In [81]:
#En este archivo ipynb vamos a desmenuzar el xlsx original para poder trabajar con la solapa 'hechos':

In [82]:
import pandas as pd
import numpy as np
import re

In [83]:
# cargamos los archivos auxiliares desde el xlsx para luego pasarlos a csv:
df_accidentes = pd.read_excel('Datasets_originales/homicidios.xlsx', sheet_name='HECHOS')

In [84]:
#para no confundirnos con las 'tablas de hechos', en vez de llamarle asi a este df le llamaremos 'accidentes'. 

In [85]:
df_accidentes_copy = df_accidentes.copy()


In [86]:
# pasamos todos los nombres de las columnas a mayúsculas:
df_accidentes_copy.columns = df_accidentes_copy.columns.str.upper()


In [87]:
# sacamos los acentos a los nombres de las columnas:
df_accidentes_copy.columns = df_accidentes_copy.columns.str.replace('Á', 'A')
df_accidentes_copy.columns = df_accidentes_copy.columns.str.replace('É', 'E')
df_accidentes_copy.columns = df_accidentes_copy.columns.str.replace('Í', 'I')
df_accidentes_copy.columns = df_accidentes_copy.columns.str.replace('Ó', 'O')
df_accidentes_copy.columns = df_accidentes_copy.columns.str.replace('Ú', 'U')
df_accidentes_copy.columns = df_accidentes_copy.columns.str.replace('Ñ', 'N')
print('Nombres de las columnas sin acentos:')
print(df_accidentes_copy.columns)
print('\n')



Nombres de las columnas sin acentos:
Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'CALLE', 'ALTURA', 'CRUCE',
       'DIRECCION NORMALIZADA', 'COMUNA', 'XY (CABA)', 'POS X', 'POS Y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')




In [88]:
#coloco el _ en el medio de las palabras compuestas:
df_accidentes_copy.columns = df_accidentes_copy.columns.str.replace(' ', '_')
print('Nombres de las columnas con _ en lugar de espacio:')
print(df_accidentes_copy.columns)
print('\n')
    

Nombres de las columnas con _ en lugar de espacio:
Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'CALLE', 'ALTURA', 'CRUCE',
       'DIRECCION_NORMALIZADA', 'COMUNA', 'XY_(CABA)', 'POS_X', 'POS_Y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')




In [89]:
# buscar letras raros en todo el dataframe:

# Crear un DataFrame vacío para almacenar los resultados
result = pd.DataFrame()

for col in df_accidentes_copy.columns:
    # Buscar valores no ASCII
    non_ascii_values = df_accidentes_copy[col].apply(lambda x: re.search(r'[^\x00-\x7F]', str(x)) is not None)
    # Crear un DataFrame con los resultados
    df = pd.DataFrame(df_accidentes_copy[col][non_ascii_values].unique(), columns=[col])
    # Añadir los resultados al DataFrame de resultados
    result = pd.concat([result, df], axis=1)

# Guardar el DataFrame de resultados en un archivo CSV
result.to_csv('non_ascii_values.csv', index=False)



In [90]:
#En el csv que nos devolvio, encontramos varios tipos de caracteres raros, casi todos relacionados con la ñ y Ñ vamos a comenzar por reemplazarlos:
df_accidentes_copy['DIRECCION_NORMALIZADA'] = df_accidentes_copy['DIRECCION_NORMALIZADA'].str.replace('Ã‘', 'Ñ')

In [91]:
#el segundoo es Ã‰ que corresponde a É:
df_accidentes_copy['DIRECCION_NORMALIZADA'] = df_accidentes_copy['DIRECCION_NORMALIZADA'].str.replace('Ã‰', 'É')

In [92]:
#El tercero es Ã, que corresponde a Á:
df_accidentes_copy['DIRECCION_NORMALIZADA'] = df_accidentes_copy['DIRECCION_NORMALIZADA'].str.replace('Ã', 'Á')

In [93]:
#Y el cuarto y ultimo es el Â° que en este caso y solo en este caso corresponde a ° A:
df_accidentes_copy['DIRECCION_NORMALIZADA'] = df_accidentes_copy['DIRECCION_NORMALIZADA'].str.replace('Â°', '° A')

In [94]:
#Haciendo una una inspección visual de la columna 'DIRECCION_NORMALIZADA' encontramos que hay un valor que no corresponde a una dirección, sino a un nombre de calle, por lo que lo vamos a reemplazar, el mismo es NU?EZ por NUÑEZ:
df_accidentes_copy['CALLE'] = df_accidentes_copy['CALLE'].str.replace('NU?EZ', 'NUÑEZ')

In [95]:
# buscamos los valores nulos en el dataset de accidentes:
print('Valores nulos en el dataset de accidentes:')
print(df_accidentes_copy.isnull().sum())
print('\n')



Valores nulos en el dataset de accidentes:
ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
CALLE                      1
ALTURA                   567
CRUCE                    171
DIRECCION_NORMALIZADA      8
COMUNA                     0
XY_(CABA)                  0
POS_X                      0
POS_Y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64




In [96]:
# vamos a eliminar las columnas que tienen mas nulos que datos:
# df_accidentes_copy = df_accidentes_copy.dropna(axis=1, thresh=0.5*len(df_accidentes))
# print('Columnas eliminadas:')
# print(df_accidentes_copy.isnull().sum())
# print('\n')



In [97]:
# vamos a eliminar las columnas 'ALTURA', 'CRUCE' Y 'XY (CABA)' ya que no aportan informacion relevante:
df_accidentes_copy = df_accidentes_copy.drop(columns=['ALTURA', 'CRUCE','XY_(CABA)'])
print('Columnas no eliminadas:')
print(df_accidentes_copy.columns)
print('\n')




Columnas no eliminadas:
Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'CALLE', 'DIRECCION_NORMALIZADA',
       'COMUNA', 'POS_X', 'POS_Y', 'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')




In [98]:
#buscar la fila con el valor nulo en la columna 'CALLE' y 'DIRECCION NORMALIZADA':
print('Fila con el valor nulo en la columna CALLE:')
print(df_accidentes_copy[df_accidentes_copy['CALLE'].isnull()])
print('\n')




Fila con el valor nulo en la columna CALLE:
            ID  N_VICTIMAS      FECHA  AAAA  MM  DD      HORA  HH  \
119  2016-0151           1 2016-11-18  2016  11  18  20:35:00  20   

    LUGAR_DEL_HECHO TIPO_DE_CALLE CALLE DIRECCION_NORMALIZADA  COMUNA POS_X  \
119              SD         CALLE   NaN                   NaN       0     .   

    POS_Y PARTICIPANTES VICTIMA ACUSADO  
119     .     PEATON-SD  PEATON      SD  




In [99]:
#VER LA fila 119 completa:
print('Fila 119 completa:')
print(df_accidentes_copy.iloc[119])
print('\n')



Fila 119 completa:
ID                                 2016-0151
N_VICTIMAS                                 1
FECHA                    2016-11-18 00:00:00
AAAA                                    2016
MM                                        11
DD                                        18
HORA                                20:35:00
HH                                        20
LUGAR_DEL_HECHO                           SD
TIPO_DE_CALLE                          CALLE
CALLE                                    NaN
DIRECCION_NORMALIZADA                    NaN
COMUNA                                     0
POS_X                                      .
POS_Y                                      .
PARTICIPANTES                      PEATON-SD
VICTIMA                               PEATON
ACUSADO                                   SD
Name: 119, dtype: object




In [100]:
#BORRAMOS la fila 119 ya que carece de demasiada informacion relevante:
df_accidentes_copy = df_accidentes_copy.drop([119])
print('Fila 119 eliminada:')
print(df_accidentes_copy.iloc[119])
print('\n')



Fila 119 eliminada:
ID                                          2016-0152
N_VICTIMAS                                          1
FECHA                             2016-11-23 00:00:00
AAAA                                             2016
MM                                                 11
DD                                                 23
HORA                                         21:10:00
HH                                                 21
LUGAR_DEL_HECHO            AV PERITO MORENO Y ZUVIRIA
TIPO_DE_CALLE                                 AVENIDA
CALLE                              MORENO, PERITO AV.
DIRECCION_NORMALIZADA    MORENO, PERITO AV. y ZUVIRIA
COMUNA                                              7
POS_X                                    -58.46451841
POS_Y                                    -34.64991946
PARTICIPANTES                               MOTO-AUTO
VICTIMA                                          MOTO
ACUSADO                                          AUTO
Name: 12

In [101]:
print('Fila con el valor nulo en la columna DIRECCION NORMALIZADA:')
print(df_accidentes_copy[df_accidentes_copy['DIRECCION_NORMALIZADA'].isnull()])
print('\n')


Fila con el valor nulo en la columna DIRECCION NORMALIZADA:
            ID  N_VICTIMAS      FECHA  AAAA  MM  DD      HORA  HH  \
38   2016-0052           1 2016-04-20  2016   4  20  20:00:00  20   
106  2016-0136           1 2016-10-25  2016  10  25  00:00:00   0   
180  2017-0050           2 2017-04-28  2017   4  28  11:08:08  11   
181  2017-0051           1 2017-05-01  2017   5   1  03:47:47   3   
313  2018-0039           1 2018-04-21  2018   4  21  22:15:00  22   
546  2020-0026           1 2020-05-17  2020   5  17  06:40:00   6   
621  2021-0023           1 2021-03-01  2021   3   1  09:20:00   9   

                             LUGAR_DEL_HECHO TIPO_DE_CALLE  \
38                AUTOPISTA LUGONES PK 10000     AUTOPISTA   
106         AU BUENOS AIRES - LA PLATA KM. 4     AUTOPISTA   
180  AU PERITO MORENO Y RAMAL ENLACE AU1/AU6     AUTOPISTA   
181                       AU DELLEPIANE 2400     AUTOPISTA   
313                 AUTOPISTA LUGONES KM 4.7     AUTOPISTA   
546            

In [102]:
df_accidentes_copy.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'CALLE', 'DIRECCION_NORMALIZADA',
       'COMUNA', 'POS_X', 'POS_Y', 'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [103]:
#este codigo verifica la cantidad de valores NaN en la columna DIRECCION_NORMALIZADA.
#Reemplaza los valores NaN en DIRECCION_NORMALIZADA con los valores de LUGAR_DEL_HECHO.
#Evalúa si DIRECCION_NORMALIZADA es redundante comparándola con LUGAR_DEL_HECHO. Si son iguales, elimina DIRECCION_NORMALIZADA.
#Guarda el DataFrame limpio en un archivo CSV.

# Verificar valores NaN en DIRECCION_NORMALIZADA
nan_count = df_accidentes_copy['DIRECCION_NORMALIZADA'].isna().sum()
print(f"Valores NaN en DIRECCION_NORMALIZADA: {nan_count}")



Valores NaN en DIRECCION_NORMALIZADA: 7


In [104]:
# Reemplazar NaN en DIRECCION_NORMALIZADA con LUGAR_DEL_HECHO si no está vacío
df_accidentes_copy['DIRECCION_NORMALIZADA'].fillna(df_accidentes_copy['LUGAR_DEL_HECHO'], inplace=True)



/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_29177/3072453504.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_accidentes_copy['DIRECCION_NORMALIZADA'].fillna(df_accidentes_copy['LUGAR_DEL_HECHO'], inplace=True)


In [105]:
# Verificar si DIRECCION_NORMALIZADA sigue siendo necesaria
if df_accidentes_copy['DIRECCION_NORMALIZADA'].equals(df_accidentes_copy['LUGAR_DEL_HECHO']):
    print("DIRECCION_NORMALIZADA es redundante y puede ser eliminada.")
    df_accidentes_copy.drop(columns=['DIRECCION_NORMALIZADA'], inplace=True)
else:
    print("DIRECCION_NORMALIZADA contiene información única y debe ser conservada.")

# Este código compara dos columnas del DataFrame df_accidentes_copy: 'DIRECCION_NORMALIZADA' y 'LUGAR_DEL_HECHO'.

# El método equals() verifica si las dos columnas son idénticas, es decir, si todos sus elementos correspondientes son iguales.

# Si las dos columnas son idénticas, el código imprime "DIRECCION_NORMALIZADA es redundante y puede ser eliminada." y luego elimina la columna 'DIRECCION_NORMALIZADA' del DataFrame utilizando el método drop(). El argumento inplace=True significa que la operación se realiza en el DataFrame original, en lugar de devolver un nuevo DataFrame.

# Si las dos columnas no son idénticas, el código imprime "DIRECCION_NORMALIZADA contiene información única y debe ser conservada.". Esto significa que la columna 'DIRECCION_NORMALIZADA' contiene al menos algún dato que no está presente en la columna 'LUGAR_DEL_HECHO', por lo que no se elimina.

DIRECCION_NORMALIZADA contiene información única y debe ser conservada.


In [106]:
# BUSCAR todas las celdas que contengan la palabra 'SD'

# Crear una máscara booleana donde True indica que la celda contiene 'SD'
mask = df_accidentes_copy.applymap(lambda x: 'SD' in str(x))

/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_29177/2400321010.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df_accidentes_copy.applymap(lambda x: 'SD' in str(x))


In [107]:
# Imprimir las filas donde al menos una celda contiene 'SD'
(df_accidentes_copy[mask.any(axis=1)]).shape

(27, 18)

In [108]:
df_accidentes_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 695
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          695 non-null    object        
 10  CALLE                  695 non-null    object        
 11  DIRECCION_NORMALIZADA  695 non-null    object        
 12  COMUNA                 695 non-null    int64         
 13  POS_X     

In [109]:
# Crear una máscara booleana donde True indica que la celda contiene 'SD'
mask = df_accidentes_copy.apply(lambda x: x.str.contains('SD').any() if x.dtype == "object" else False)

# Imprimir las columnas donde al menos una celda contiene 'SD'
print(df_accidentes_copy.columns[mask])

Index(['HORA', 'HH', 'PARTICIPANTES', 'VICTIMA', 'ACUSADO'], dtype='object')


In [110]:
#mostrar las filas que contienen SD en la columna 'HORA','HH','PARTICIPANTES','VICTIMA', 'ACUSADO'
print('Fila con SD en la columna HORA:')
print(df_accidentes_copy[df_accidentes_copy['HORA'].str.contains('SD', na=False)])
print('\n')

Fila con SD en la columna HORA:
            ID  N_VICTIMAS      FECHA  AAAA  MM  DD HORA  HH  \
518  2019-0103           1 2019-12-18  2019  12  18   SD  SD   

             LUGAR_DEL_HECHO TIPO_DE_CALLE           CALLE  \
518  PAZ, GRAL. AV. Y GRIVEO      GRAL PAZ  PAZ, GRAL. AV.   

       DIRECCION_NORMALIZADA  COMUNA         POS_X         POS_Y  \
518  PAZ, GRAL. AV. y GRIVEO      11  -58.52169422  -34.59471640   

    PARTICIPANTES VICTIMA ACUSADO  
518     MOTO-MOTO    MOTO    MOTO  




In [111]:
# Calcular la mediana de la columna 'HORA'

from datetime import datetime, time

# Función para convertir un string a un tiempo
def str_a_tiempo(s):
    try:
        return datetime.strptime(s, '%H:%M:%S').time()
    except ValueError:
        return None

# Convertir los strings en la columna 'HORA' a objetos datetime.time
df_accidentes_copy['HORA'] = df_accidentes_copy['HORA'].apply(lambda x: str_a_tiempo(x) if isinstance(x, str) else x)

# Convertir los tiempos en la columna 'HORA' a segundos desde la medianoche
segundos_desde_la_medianoche = df_accidentes_copy['HORA'].dropna().apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

# Calcular la mediana de los segundos
mediana_segundos = segundos_desde_la_medianoche.median()

# Convertir la mediana de segundos de vuelta a un tiempo
mediana_HORA = time(int(mediana_segundos // 3600), int((mediana_segundos % 3600) // 60), int(mediana_segundos % 60))

# Reemplazar los NaN en la columna 'HORA' con la mediana
df_accidentes_copy['HORA'] = df_accidentes_copy['HORA'].fillna(mediana_HORA)

In [112]:
#revisamos si ha quedado SD en la columna 'HORA'
print('Fila con SD en la columna HORA:')
print(df_accidentes_copy[df_accidentes_copy['HORA'].str.contains('SD', na=False)])
print('\n')

#solucionado entonces el problema de SD en la columna 'HORA'

Fila con SD en la columna HORA:
Empty DataFrame
Columns: [ID, N_VICTIMAS, FECHA, AAAA, MM, DD, HORA, HH, LUGAR_DEL_HECHO, TIPO_DE_CALLE, CALLE, DIRECCION_NORMALIZADA, COMUNA, POS_X, POS_Y, PARTICIPANTES, VICTIMA, ACUSADO]
Index: []




In [113]:
# Calcular la mediana de la columna 'HH'
import numpy as np

# Reemplazar 'SD' con NaN
df_accidentes_copy['HH'] = df_accidentes_copy['HH'].replace('SD', np.nan)

# Calcular la mediana de la columna 'HH'
mediana_HH = df_accidentes_copy['HH'].astype(float).median()

# Reemplazar los NaN en la columna 'HH' con la mediana
df_accidentes_copy['HH'] = df_accidentes_copy['HH'].fillna(mediana_HH)

# Convertir la columna 'HH' a int
df_accidentes_copy['HH'] = df_accidentes_copy['HH'].astype(int)



/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_29177/2442409241.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_accidentes_copy['HH'] = df_accidentes_copy['HH'].replace('SD', np.nan)


In [114]:
# Convertir la columna 'HH' a string
df_accidentes_copy['HH'] = df_accidentes_copy['HH'].astype(str)

# Revisar si ha quedado 'SD' en la columna 'HH'
print('Fila con SD en la columna HH:')
print(df_accidentes_copy[df_accidentes_copy['HH'].str.contains('SD', na=False)])
print('\n')

Fila con SD en la columna HH:
Empty DataFrame
Columns: [ID, N_VICTIMAS, FECHA, AAAA, MM, DD, HORA, HH, LUGAR_DEL_HECHO, TIPO_DE_CALLE, CALLE, DIRECCION_NORMALIZADA, COMUNA, POS_X, POS_Y, PARTICIPANTES, VICTIMA, ACUSADO]
Index: []




In [115]:
#volvemos a convertir la columna 'HH' a int
df_accidentes_copy['HH'] = df_accidentes_copy['HH'].astype(int)



In [116]:
print(df_accidentes_copy[df_accidentes_copy['PARTICIPANTES'].str.contains('SD', na=False)]['PARTICIPANTES'].shape)#sin el shape nos muestra la columna completa
# No la vamos a tomar como un problema ya que no es relevante para el análisis de los datos, ademas lo usaremos para analizar los Sin Datos en la columna 'VICTIMA' y 'ACUSADO'

(26,)


In [117]:
print('Fila con SD en la columna VICTIMA:')
print(df_accidentes_copy[df_accidentes_copy['VICTIMA'].str.contains('SD', na=False)].shape)#sin el shape nos muestra la columna completa
print('\n')

# No la vamos a tomar como un problema ya que no es relevante para el análisis de los datos, ademas lo usaremos para analizar los Sin Datos en la columna 'PARTICIPANTES' y 'ACUSADO'

Fila con SD en la columna VICTIMA:
(9, 18)




In [118]:
print('Fila con SD en la columna ACUSADO:')
print(df_accidentes_copy[df_accidentes_copy['ACUSADO'].str.contains('SD', na=False)].shape)#sin el shape nos muestra la columna completa
print('\n')

# No la vamos a tomar como un problema ya que no es relevante para el análisis de los datos, ademas lo usaremos para analizar los Sin Datos en la columna 'PARTICIPANTES' y 'VICTIMA'

Fila con SD en la columna ACUSADO:
(22, 18)




In [119]:
#revisar nuevamente si han quedado nulls en el dataset de accidentes:
print('Valores nulos en el dataset de accidentes:')
print(df_accidentes_copy.isnull().sum())
print('\n')

Valores nulos en el dataset de accidentes:
ID                       0
N_VICTIMAS               0
FECHA                    0
AAAA                     0
MM                       0
DD                       0
HORA                     0
HH                       0
LUGAR_DEL_HECHO          0
TIPO_DE_CALLE            0
CALLE                    0
DIRECCION_NORMALIZADA    0
COMUNA                   0
POS_X                    0
POS_Y                    0
PARTICIPANTES            0
VICTIMA                  0
ACUSADO                  0
dtype: int64




In [120]:
#Con todo el DF limpio y ordenado, agregamos la columna 'FRANJA_HORARIA' para luego hacer ahalisis por rangos horarios:
#NOTA IMPORTANTE: en los 'bins' se debe colocar -1 y25 como limites para que los datos se ajusten correctamente, ya que si se coloca 0 y 24, los datos no se ajustan correctamente, y el resultado es un Nan.-

df_accidentes_copy['FRANJA_HORARIA'] = pd.cut(df_accidentes_copy['HH'], bins=[-1, 6, 12, 18, 25], labels=['Madrugada', 'Mañana', 'Tarde', 'Noche'])

In [121]:
#revisamos nuevamente si han quedado nulls en el dataset de accidentes:
print('Valores nulos en el dataset de accidentes:')
print(df_accidentes_copy.isnull().sum())
print('\n')

Valores nulos en el dataset de accidentes:
ID                       0
N_VICTIMAS               0
FECHA                    0
AAAA                     0
MM                       0
DD                       0
HORA                     0
HH                       0
LUGAR_DEL_HECHO          0
TIPO_DE_CALLE            0
CALLE                    0
DIRECCION_NORMALIZADA    0
COMUNA                   0
POS_X                    0
POS_Y                    0
PARTICIPANTES            0
VICTIMA                  0
ACUSADO                  0
FRANJA_HORARIA           0
dtype: int64




In [122]:
#MOVEMOS la columna 'FRANJA_HORARIA' al lado de 'HH' para que quede mas ordenado:

# Extraer la columna 'FRANJA_HORARIA'
franja_horaria = df_accidentes_copy.pop('FRANJA_HORARIA')

In [123]:
# Insertar la columna 'FRANJA_HORARIA' en la posición deseada
df_accidentes_copy.insert(df_accidentes_copy.columns.get_loc('HH') + 1, 'FRANJA_HORARIA', franja_horaria)

In [128]:
# Guardamos el DataFrame limpio listo para usar en un archivo xlsx:
df_accidentes_copy.to_excel('Datasets_limpios/df_hechos_ETL.xlsx', index=False)

In [129]:
# Guardamos el DataFrame limpio listo para usar en un archivo CSV:
df_accidentes_copy.to_csv('Datasets_limpios/df_hechos_ETL.csv', index=False, encoding='utf-8')